# Setup

In [ ]:
# Imports
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pirl
from pirl.experiments import plots as myplots

In [ ]:
fname = 'data/few-jungle-20180326_163501-7893bc.pkl'
data = pd.read_pickle(fname)

# Value difference

In [ ]:
def plot_value(data, kind, algo_pattern='(.*)', env_pattern='(.*)', algos=['.*'], dps=2):
    value = myplots.extract_value(data)[kind]
    value.items.name = 'Algorithm'
    value.major_axis.name = 'Environment'
    value.minor_axis.name = 'Trajectories'
    value.items = value.items.str.extract(algo_pattern, expand=False)
    value.major_axis = value.major_axis.str.extract(env_pattern, expand=False)
    
    matches = []
    mask = pd.Series(False, index=value.items)
    for p in algos:
        m = value.items.str.match(p)
        matches += list(value.items[m & (~mask)])
        mask |= m
    value = value.loc[matches, :, :]
    
    value.items = value.items.str.split('_').str.join(' ')  # so lines wrap
    value = value.to_frame()
    value = value.round(dps)
    return value
    
value_kind = 'optimal'
algo_pattern = '(.*)'
env_pattern = 'pirl/GridWorld-Jungle-9x9-(.*)-v0'
plot_value(data, value_kind, algo_pattern, env_pattern, ['ground_truth', 'mces', 'mcep.*'])

# Visualizing rewards (gridworld only)

In [ ]:
shape = (9,9)
irl_algo = 'mces'
figs = myplots.gridworld_heatmap(data['reward'][irl_algo], shape)
for fig in figs:
    display(fig[1])

In [ ]:
# Save to disk
out_dir = './figs/some-experiment'
for k, v in data['reward'].items():
    pirl.experiments.plots.save_figs(pirl.experiments.plots.gridworld_heatmap(v, (4,4)), os.path.join(out_dir, k))